In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
categorical_var = ["university", "sex", "gender_identity", "sexual_orientation", "ethnic_group", 'marital_status', 'student_accommodation', "work", 'RENDA_categorias', 'student_of', 'sedentary_behavior', "sedentary_2", "phq9", "phq9_severity", "gad7", "gad7_severity"]
db = pd.read_excel("BANCO_LIMPO_INT.xlsx")
#for var in categorical_var:
#    print(f"Response counts for {var}:")
#    print(db[var].value_counts())
#    print() 

creating contingency tables for every variable: 

In [ ]:
with pd.ExcelWriter('crosstab_variables.xlsx') as writer:    
    for col1 in categorical_var:
        for col2 in categorical_var:
            if col1 != col2: 
                cont = pd.crosstab(db[col1], db[col2])
                cont.loc[''] = cont.sum()
                cont[''] = cont.sum(axis=1)
                cont = cont.reset_index()
                cont.rename(columns={col1: ''}, inplace=True)
                sheet_name = f'{col1}_vs_{col2}'[:30]
                cont.to_excel(writer, sheet_name=sheet_name, index=False)
                
        

testing for chi-squared and cramer's v:

In [1]:
xlsx_file = pd.ExcelFile('crosstab_variables.xlsx')
for sheet_name in xlsx_file.sheet_names:
    contingency_table = pd.read_excel(xlsx_file, sheet_name = sheet_name)
    contingency_table = contingency_table.fillna(0)
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    n = contingency_table.to_numpy().sum()
    v_cramer = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
    chi_squared_results = pd.DataFrame({
                    'Estatística': ['Chi-quadrado', 'valor p', 'Graus de liberdade', "V de Cramer"],
                    'Valor': [chi2, p, dof, v_cramer]
                })
    print(" ")
    print(sheet_name)
    print(chi_squared_results)
    print(" ")
    if p < 0.05: 
        print("Hipótese nula rejeitada; há dependência das variáveis") 
        if v_cramer < 0.3: print("Dependência fraca entre as variáveis; ")
        elif v_cramer > 0.6: print("Dependência forte entre variáveis")
        else: print("Dependência razoável entre variáveis")
    else: print("Hipótese alternativa rejeitada; variáveis independentes")


NameError: name 'pd' is not defined